<h3>Dependency Installation<h3>

In [2]:
! pip install pypots==0.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.2/497.2 kB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


<h3>Importando o pypots<h3>

In [1]:
import pypots
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed

2024-11-28 14:19:37.050643: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.8/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.8/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


 <h3>Preparing the PhysioNet-2012 dataset for this tutorial<h3>

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = benchpots.datasets.preprocess_physionet2012(subset="all", rate=0.1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2024-11-28 14:19:40 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2024-11-28 14:19:40 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-11-28 14:19:40 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-11-28 14:19:40 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-11-28 14:19:41 [INFO]: Loaded successfully!
2024-11-28 14:19:59 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2024-11-28 14:19:59 [INFO]: 68807 values masked out in the val set as ground truth, take 9.97% of the original observed values
2024-11-28 14:19:59 [INFO]: 86319 values masked out in the test set as ground truth, take 9.99% of the original observed v

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


<h3>Imputation Models<h3>

In [3]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}
# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
}
## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = np.isnan(physionet2012_dataset['test_X_ori']) ^ np.isnan(physionet2012_dataset['test_X'])
test_X_ori = np.nan_to_num(physionet2012_dataset['test_X_ori'])  # metric functions do not accpet input with NaNs, hence fill NaNs with 0

<h3>An example of SAITS for imputation<h3>

<h4>Initialize the model<h4>

In [4]:
# initialize the model
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)


2024-11-28 14:20:15 [INFO]: No given device, using default device: cpu
2024-11-28 14:20:15 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20241128_T142015
2024-11-28 14:20:15 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20241128_T142015/tensorboard
2024-11-28 14:20:15 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Train the model<h4>

In [12]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2024-11-18 11:56:45 [INFO]: Epoch 001 - training loss: 0.7230, validation loss: 6.7596
2024-11-18 11:56:45 [INFO]: Saved the model to tutorial_results/imputation/saits/20241118_T115553/SAITS_epoch1_loss6.75964179734389.pypots
2024-11-18 11:57:10 [INFO]: Epoch 002 - training loss: 0.5370, validation loss: 6.7420
2024-11-18 11:57:10 [INFO]: Saved the model to tutorial_results/imputation/saits/20241118_T115553/SAITS_epoch2_loss6.741982015470664.pypots
2024-11-18 11:57:38 [INFO]: Epoch 003 - training loss: 0.4953, validation loss: 6.7242
2024-11-18 11:57:38 [INFO]: Saved the model to tutorial_results/imputation/saits/20241118_T115553/SAITS_epoch3_loss6.724190835654736.pypots
2024-11-18 11:58:06 [INFO]: Epoch 004 - training loss: 0.4641, validation loss: 6.7040
2024-11-18 11:58:06 [INFO]: Saved the model to tutorial_results/imputation/saits/20241118_T115553/SAITS_epoch4_loss6.703969217836857.pypots
2024-11-18 11:58:32 [INFO]: Epoch 005 - training loss: 0.4387, validation loss: 6.6843
2024-1

In [5]:
saits.load('/data/rayssa/missingdata/tutorial_results/imputation/saits/20241124_T223952/SAITS.pypots')

/usr/local/lib/python3.8/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2024-11-28

<h4> The testing stage <h4>

In [13]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
saits_results = saits.predict(dataset_for_testing)
saits_imputation = saits_results["imputation"]

<h4> Calculate mean absolute error <h4>

In [14]:
# calculate mean absolute error on the ground truth (artificially-missing values)
testing_mae = calc_mae(
    saits_imputation,
    test_X_ori,
    test_X_indicating_mask,
)
print(f"Testing mean absolute error: {testing_mae:.4f}")

Testing mean absolute error: 0.2458
